In [2]:
#Import dependencies
import torch
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Get data
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)

In [4]:
#Image Classifier NN
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 63, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)
    )
    def forward(self, x):
        return self.model(x)


In [5]:
#Instance of neural network
# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Instance of neural network
clf = ImageClassifier().to(device)
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [6]:
#Training flow

if __name__ == "__main__":
    for epoch in range(10): 
        for batch in dataset:
            X,y = batch
            X,y = X.to(device), y.to(device)
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            # Backpropagation applied
            opt.zero_grad()
            loss.backward()
            opt.step()

        print(f"Epoch:{epoch} loss is {loss.item()}")

with open('model_state.pt', 'wb') as f:
    save(clf.state_dict(), f)

RuntimeError: linear(): input and weight.T shapes cannot be multiplied (32x30492 and 30976x10)